# FIT5196 Task 2 in Assessment 1
#### Student Name: Sai Ram D Gitte
#### Student ID: 31009751

Date: 13/09/2020

Version: 1.0

Environment: Python 3.7.4 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe) 
* re (for regular expression) 
* langid (for language classification) 
* numpy (for numpy array) 
* nltk (for natural language processing)


In [ ]:
# Importing Libraries 
import pandas as pd 
import langid 
import numpy as np
import re
from nltk.probability import *

# Reading the file and its sheets
- reading the xlsx file in pandas 
- cleaning the header, dropping all the Nan's and NAs

In [ ]:
sheet_list_df = pd.read_excel("31009751.xlsx", sheet_name = None)

In [ ]:
# Iterating through each dataframe in sheet_list_df and to get rid of all the 'nan' is all the columns and rows
list_of_dfs = []

for key, value in sheet_list_df.items():
    
        sheet_list_df_2 = sheet_list_df[key].dropna(axis=0, how="all")
        sheet_list_df_3 = sheet_list_df_2.dropna(axis=1, how="all")


        if ~np.isin(['text'], sheet_list_df_3.columns):
            # Cleaning the header part of each dataframe 
            # store 'text','id', and 'created_at' row to make it the header
            new_header = sheet_list_df_3.iloc[0]

            # Select all the rows below the header to be the rows of the data frame
            sheet_list_df_3 = sheet_list_df_3[1:]                             

            # Place the stored header name as the header for the new dataframe
            sheet_list_df_3.columns = new_header

            # Store all the dataframes as an element in a list
            list_of_dfs.append(sheet_list_df_3)
            
        else:
            sheet_list_df_2 = sheet_list_df[key].dropna(axis=0, how="all")
            sheet_list_df_3 = sheet_list_df_2.dropna(axis=1, how="all")
            
            list_of_dfs.append(sheet_list_df_3)

# Language classification and tokenization
- determining the language of each text
- retaining only the english tweets
- tokenizing all the texts 
- takes about 20mins to run

In [ ]:
# Retaining only the english tweets classified by langid
for i in range(len(list_of_dfs)):
    list_of_dfs[i]['language'] = list_of_dfs[i]['text'].apply(lambda x: langid.classify(x)[0] if not str(x).isdigit() else None)
    list_of_dfs[i] = list_of_dfs[i][list_of_dfs[i]['language'] == 'en']
    list_of_dfs[i]['tokens'] = list_of_dfs[i]['text'].apply(lambda x: str(re.findall("[a-zA-Z]+(?:[-'][a-zA-Z]+)?", x.lower())))

### Getting the overall words used in all of the sheets in just one list 

In [ ]:
import itertools
token_list = []

for i in list_of_dfs:
    sub_token_list = []
    for j in range(len(i['tokens'])):
        sub_token_list.append(eval(i['tokens'].values[j]))
    token_list.append(list(itertools.chain.from_iterable(sub_token_list)))   


In [ ]:
# Flattening the list of lists and having a copy stored 
all_words = list(itertools.chain.from_iterable(token_list))
all_words_dup = list(itertools.chain.from_iterable(token_list))

# Context independent removal 
- reading the stop words file
- getting rid of all the independent stop words from the flattened list


In [ ]:
# Removing context independent words
stop_words = pd.read_csv("stopwords_en.txt", names=['words'])

for word in stop_words['words'].values:
    if word in all_words:
        all_words_dup.pop(all_words_dup.index(word))

# Putting pandas to good use!!
### To eleminate context dependent words from the doc
- reading each day's tokenized words into columns
- having a separate dataframe to hold the words and their repsective counts
- matching the words in the dataframe to each columns of the main dataframe and then counting the frequency

In [ ]:
actual_df = pd.DataFrame(token_list).transpose()

In [ ]:
all_words_df_count = pd.DataFrame(set(all_words_dup))
all_words_df_count['count'] = [0 for i in range(all_words_df_count.shape[0])]
all_words_df_count = all_words_df_count.set_index(0)

In [ ]:
for i in actual_df.columns:
    matching_words = np.intersect1d(all_words_df_count.index.values, np.unique(actual_df[i].values[actual_df[i].values != np.array(None)]), return_indices = True )[1]
    all_words_df_count.iloc[matching_words,:] += 1
all_words_df_count

### Filtering out the context dependent words and the rare tokens simultaneously

In [ ]:
context_dependent_words = all_words_df_count[all_words_df_count['count'] > 60]
filtered_df = all_words_df_count[all_words_df_count['count'] <= 60] # Conext dependent words removal
filtered_df = filtered_df[filtered_df['count'] >= 5] # Rare tokens removal

# Bigrams from the entire document
- nltk package is used to determine the top 200 bigrams from the corpus
- any bigram containing independent stop word is not included
- extracting the bigrams and their frequncy counts

In [ ]:
import nltk
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words_dup)
bigram_finder.apply_freq_filter(70)
bigram_finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stop_words['words'].values)

top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-200 bigrams
top_200_bigrams


In [ ]:
# Get the count of each bigram from the top 200 bigrams we saw above
bigrams_count = []
for i in top_200_bigrams:
    for key,value in dict(bigram_finder.ngram_fd.items()).items():  
        if i == key:
            bigrams_count.append(key[0]+"_"+key[1]+":"+str(value))

### Using dataframe to hold the word count 
- each word is taken in a row
- each word is stemmed using porter stemmer
- list of string of each word and its freq is stored

In [ ]:
from nltk.stem import PorterStemmer
port_stem = PorterStemmer()


# dup_df = pd.DataFrame(columns= ['word'])
filtered_df['word'] = filtered_df.index.values
filtered_df['stemmed'] = filtered_df['word'].apply(lambda x: port_stem.stem(x))
single_words = filtered_df['stemmed'].values+ np.array([':']) + filtered_df['count'].values.astype(str).astype('object')
single_words

- the words and their counts are taken in two diff columns
- the repeating words are groupedby and their counts are summed

In [ ]:
vocab = sorted(list(single_words) + bigrams_count)

# To sum up the repeating word's count
dffff = pd.DataFrame(vocab)
dffff['count'] = dffff[0].apply(lambda x: int(x.split(':')[1]))
dffff[0] = dffff[0].apply(lambda x: (x.split(':')[0]))
dffff = dffff.groupby([0]).sum()
dffff['length'] = dffff.reset_index()[0].apply(lambda x: len(x)).values
dffff = dffff[dffff['length'] >= 3]
total_words_count = dffff.index.values+ np.array([':']) + dffff['count'].values.astype(str).astype('object')
total_words_count

# Writing the data into a txt file

In [ ]:
f = open("31009751_vocab.txt", "w")
for i in total_words_count:
    f.write(i +'\n')
f.close()

# Sparse Matrix
- for each of the tokenized words calculating its count and then repeating itself

In [ ]:
# Sparse matrix calculation logic for the entire document
vfu = np.vectorize(lambda t: str(t+' ') )
stri = vfu(filtered_df.index.values) 
sparse_words = stri.astype('object') * filtered_df['count'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(sparse_words)

print(X)

# Task 2 .2
- Bigrams and unigrams for each day

In [ ]:
# Context Independent Words Removal 

duplicate = [j for j in token_list]

for stop_word in stop_words['words'].values:
    for k in range(len(duplicate)):
        if stop_word in token_list[k]:
            duplicate[k].pop(duplicate[k].index(stop_word))

In [ ]:
# Context dependent words removal 
list_df_words = []
for i in range(len(duplicate)):
    compare_df = pd.DataFrame(set(duplicate[i]))
    compare_df['count'] = [0 for k in range(compare_df.shape[0])]
    compare_df = compare_df.set_index(0)
    
    for j in range(len(duplicate)):
        matching = np.intersect1d(compare_df.index.values, np.unique(np.array(duplicate[j])[np.array(duplicate[j]) != np.array(None)]), assume_unique = True, return_indices = True )[1]
        compare_df.iloc[matching,:] += 1
    list_df_words.append(compare_df)

In [ ]:
# Rare tokens and context dependent words removal
for i in range(len(list_df_words)):
    list_df_words[i] = list_df_words[i][list_df_words[i]['count'] <= 60]
    list_df_words[i] = list_df_words[i][list_df_words[i]['count'] >= 5]


In [ ]:
import nltk
bigram_per_day = []
for i in range(len(token_list)):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(token_list[i])
    bigram_finder.apply_freq_filter(3)
    bigram_finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stop_words['words'].values)

    top_100_bigrams = bigram_finder.nbest(bigram_measures.pmi, 100) # Top-100 bigrams
    bigram_per_day.append(list(bigram_finder.ngram_fd.items()))


In [ ]:
# Get the sheets names
sheets = pd.ExcelFile("31009751.xlsx")
sheets = sheets.sheet_names
bigram_count_dict = dict(zip(sheets, bigram_per_day))

# Writing the data to a txt file

In [ ]:
f = open('31009751_100bi.txt', 'w')
for key,value in bigram_count_dict.items():
    f.write(key +":"+ str(value)+'\n')
f.close()

# Top 100 unigram per day

In [ ]:
from nltk.probability import FreqDist
from nltk.util import ngrams


unigram_list = []
for i in range(len(token_list)):
    bigrams = ngrams(token_list[i], 1)
    
    dict_of_bigrams = dict(FreqDist(bigrams))
    
    unigram_per_day = dict(sorted(dict_of_bigrams.items(), key=lambda x: x[1], reverse=True))
    
    unigram_without_stop = [(port_stem.stem(key[0]),value) for key,value in unigram_per_day.items() if len(key[0]) >= 3 if not any(stop in key for stop in stop_words['words'].values)]
    unigram_list.append(unigram_without_stop[0:100])


In [ ]:
unigram_count_daywise = dict(zip(sheets,unigram_list))

In [ ]:
f = open('31009751_100uni.txt', 'w')
for key,value in unigram_count_daywise.items():
    f.write(str(key) +":"+ str(value)+'\n')
f.close()

# Task 2 .3 Sparse Matrix per day

In [ ]:
# Sparse matrix calculation logic starts from here
vfu = np.vectorize(lambda t: str(t+' ') )
stri = vfu(filtered_df.index.values) 
sparse_words = stri.astype('object') * filtered_df['count'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(sparse_words)

print(X)

In [ ]:
for columns in range(len(list_df_words)):
    
    
    list_df_words[columns]['stem'] = list_df_words[columns].index.values
    list_df_words[columns]['stem'] = list_df_words[columns]['stem'].apply(lambda x: port_stem.stem(x))
    list_df_words[columns]['length'] = list_df_words[columns]['stem'].apply(lambda x: len(x))
    list_df_words[columns] = list_df_words[columns][list_df_words[columns]['length'] >= 3]


In [ ]:
# Sparse matrix calculation logic starts from here
total_sparse_words = []
vfu = np.vectorize(lambda t: str(t+' ') )

for i in list_df_words:
    
    stri = vfu(i['stem'].values) 
    sparse_words = stri.astype('object') * i['count'].values
    total_sparse_words.append(sparse_words)

# Writing the data to a txt file

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
f = open('31009751_CountVect.txt', 'w')
for k in total_sparse_words:
    X = vectorizer.fit_transform(k)

for m in sheets:
    f.write(m + ",")
    for item in X.toarray():
        if len(item[item.nonzero()]) == 1:
            f.write(str(int(item.nonzero()[0])) + ":" + str(int(item[item.nonzero()]))+",")
            
        else:
            f.write(str(item.nonzero()[0][0]) + ":" + str(item[item.nonzero()][0])+",")
            f.write(str(item.nonzero()[0][1]) + ":" + str(item[item.nonzero()][1])+",")
